## Import Libraries

In [1]:
# %matplotlib ipympl
# %matplotlib inline
%matplotlib wx

In [2]:
import matplotlib.pyplot as plt
plt.ion()

In [3]:
from pydgilib_extra import *
from atprogram.atprogram import atprogram

In [4]:
from os import getcwd, path, pardir

## Compile and program project

In [5]:
project_path = [path.curdir, "AES_Flash-S"]
project_path

['.', 'AES_Flash-S']

In [79]:
atprogram(path.abspath(path.join(*project_path)), verbose=2)

Building file: .././main.c
Invoking: ARM/GNU C Compiler : 6.3.1
"C:\Program Files (x86)\Atmel\Studio\7.0\toolchain\arm\arm-gnu-toolchain\bin\arm-none-eabi-gcc.exe"  -x c -mthumb -D__SAML11E16A__ -DDEBUG  -I"C:\Program Files (x86)\Atmel\Studio\7.0\Packs\arm\cmsis\5.0.1\CMSIS\Include" -I"../Config" -I".." -I"../examples" -I"../hal/include" -I"../hal/utils/include" -I"../hpl/core" -I"../hpl/crya" -I"../hpl/dmac" -I"../hpl/gclk" -I"../hpl/mclk" -I"../hpl/nvmctrl" -I"../hpl/osc32kctrl" -I"../hpl/oscctrl" -I"../hpl/pm" -I"../hpl/port" -I"../hri" -I"../trustzone" -I"../../../../shared" -I"C:\Program Files (x86)\Atmel\Studio\7.0\Packs\Atmel\SAML11_DFP\1.0.109\include"  -O1 -ffunction-sections -mlong-calls -g3 -Wall -mcpu=cortex-m23 -c -std=gnu99 -mcmse -MD -MP -MF "main.d" -MT"main.d" -MT"main.o"   -o "main.o" ".././main.c" 
Finished building: .././main.c
Building target: AES_Flash-S.elf
Invoking: ARM/GNU Linker : 6.3.1
"C:\Program Files (x86)\Atmel\Studio\7.0\toolchain\arm\arm-gnu-toolchain\b

0

## Data Logging

In [61]:
live_plot = False

Create a figure for the plot.

In [62]:
if live_plot:
    fig = plt.figure(figsize=(10, 6))
    fig.show()

Create the configuration dictionary for `DGILibExtra`.

In [63]:
config_dict = {
    "interfaces": [INTERFACE_POWER, INTERFACE_GPIO],
    "loggers": [LOGGER_OBJECT, LOGGER_CSV],
    "gpio_delay_time" : 0.0007,
    "file_name_base": "experiment_aes_flash"
}
config_dict_plot = {
    "loggers": [LOGGER_OBJECT, LOGGER_PLOT, LOGGER_CSV],
    "plot_pins": [False, False, True, True],
    "plot_pins_method": "line",
    "plot_xmax": 5,
    "window_title": "Experiment AES Flash",
}

Stop criteria to pass to the logger:

In [64]:
def stop_fn(logger_data):
    return all(logger_data.gpio.values[-1])

Perform the measurement.

In [80]:
data = []
cd = config_dict.copy()
if live_plot:
    fig.clf()
    for ax in fig.get_axes():
        ax.cla()
    
    cd.update(config_dict_plot)
    cd["fig"] = fig
    
with DGILibExtra(**cd) as dgilib:
    dgilib.device_reset()
    dgilib.logger.log(1000,stop_fn)
    data = dgilib.data

# Store Data

In [81]:
import pickle
pickle.dump(data, open("aes_flash_logger_data.p", "wb"))

# Load Data

In [82]:
data = pickle.load(open("aes_flash_logger_data.p", "rb"))

## Analysis

Create Stop Function to stop parsing the data when all pins are high.

In [83]:
def stop_function(pin_values):
    return all(pin_values)

Parse the data.

In [84]:
aes_charge, aes_time = power_and_time_per_pulse(data, 2, stop_function=stop_function)

In [85]:
flash_charge, flash_time = power_and_time_per_pulse(data, 3, stop_function=stop_function)

In [86]:
length = len(aes_charge)
assert length == len(aes_time)
assert length == len(flash_charge)
assert length == len(flash_time)
print(length)

750


In [87]:
aes_encrypt_charge = aes_charge[0::2]
aes_decrypt_charge = aes_charge[1::2]
aes_encrypt_time = aes_time[0::2]
aes_decrypt_time = aes_time[1::2]

In [88]:
aes_flash_write_charge = flash_charge[0::2]
aes_flash_read_charge = flash_charge[1::2]
aes_flash_write_time = flash_time[0::2]
aes_flash_read_time = flash_time[1::2]

In [89]:
length = len(aes_encrypt_charge)
assert length == len(aes_decrypt_charge)
assert length == len(aes_encrypt_time)
assert length == len(aes_decrypt_time)
assert length == len(aes_flash_write_charge)
assert length == len(aes_flash_read_charge)
assert length == len(aes_flash_write_time)
assert length == len(aes_flash_read_time)
print(length)

375


In [90]:
MBEDTLS_AES_BLOCK_SIZE = 16
STEP_SIZE = MBEDTLS_AES_BLOCK_SIZE
MIN_NUM_BYTES = STEP_SIZE
num_bytes = range(MIN_NUM_BYTES, STEP_SIZE * (MIN_AES_BLOCKS + len(aes_encrypt_charge)), STEP_SIZE)
# print(f"MAX_AES_BLOCKS: {MIN_AES_BLOCKS + len(aes_encrypt_charge) - 1}")
print(f"MAX_NUM_BYTES: {num_bytes[-1]}")

MAX_NUM_BYTES: 6000


In [91]:
from lmfit import Model

def line(x, slope, intercept):
    """a line"""
    return [slope*i + intercept for i in x]

mod = Model(line)
pars = mod.make_params(slope=0, intercept=1)

In [92]:
results = []
for y in [aes_encrypt_charge, aes_decrypt_charge, aes_encrypt_time, aes_decrypt_time, aes_flash_write_charge, aes_flash_read_charge, aes_flash_write_time, aes_flash_read_time]:
    result = mod.fit(y, pars, x=num_bytes)
    print(result.fit_report())
    fig, grid = result.plot()
#     xlabel=f"{labels[result_type]['x']} " +
#            f"[{labels[result_type]['x_unit']}]",
#     ylabel=f"{labels[result_type]['y']} " +
#            f"[{labels[result_type]['y_unit']}]")
#     fig.canvas.set_window_title(
#         f"Residuals of {parameter_name}")
    fig.tight_layout()
    fig.show()
    results.append(result)

[[Model]]
    Model(line)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 7
    # data points      = 375
    # variables        = 2
    chi-square         = 5.5316e-12
    reduced chi-square = 1.4830e-14
    Akaike info crit   = -11938.8038
    Bayesian info crit = -11930.9499
[[Variables]]
    slope:      3.2198e-08 +/- 3.6307e-12 (0.01%) (init = 0)
    intercept: -1.5492e-07 +/- 1.2602e-08 (8.13%) (init = 1)
[[Correlations]] (unreported correlations are < 0.100)
    C(slope, intercept) = -0.867

[[Model]]
    Model(line)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 7
    # data points      = 375
    # variables        = 2
    chi-square         = 3.8153e-11
    reduced chi-square = 1.0229e-13
    Akaike info crit   = -11214.6271
    Bayesian info crit = -11206.7732
[[Variables]]
    slope:      3.3574e-08 +/- 9.5353e-12 (0.03%) (init = 0)
    intercept: -1.3165e-07 +/- 3.3097e-08 (25.14%) (init = 1)
[[Correlations]] (unrepo

In [108]:
fig2 = plt.figure(figsize=(8, 6))
fig2.canvas.set_window_title("Analysis AES Flash")

In [109]:
charge_color = 'r'
time_color = 'b'

In [110]:
fig2.clf()
# fig2.suptitle("Energy analysis of AES")
ax1 = fig2.add_subplot(1, 1, 1)
ax2 = ax1.twinx()
ax1.set_xlabel('Number of bytes')
ax1.set_ylabel('Charge [C]', color=charge_color)
ax2.set_ylabel('Time [s]', color=time_color)
ax1.tick_params('y', colors=charge_color)
ax2.tick_params('y', colors=time_color)

In [111]:
lines = []
lines += ax1.plot(num_bytes, aes_encrypt_charge, charge_color+'-', label='AES Encrypt Charge')
lines += ax1.plot(num_bytes, aes_decrypt_charge, charge_color+'--', label='AES Decrypt Charge')
lines += ax2.plot(num_bytes, aes_encrypt_time, time_color+'-', label='AES Encrypt Time')
lines += ax2.plot(num_bytes, aes_decrypt_time, time_color+'--', label='AES Decrypt Time')
lines += ax1.plot(num_bytes, aes_flash_write_charge, charge_color+'-.', label='Flash Write Charge')
lines += ax1.plot(num_bytes, aes_flash_read_charge, charge_color+':', label='Flash Read Charge')
lines += ax2.plot(num_bytes, aes_flash_write_time, time_color+'-.', label='Flash Write Time')
lines += ax2.plot(num_bytes, aes_flash_read_time, time_color+':', label='Flash Read Time')
ax1.legend(handles=lines)
# [aes_encrypt_charge, aes_decrypt_charge, aes_encrypt_time, aes_decrypt_time, flash_write_charge, flash_read_charge, flash_write_time, flash_read_time]:
ax1.set_title(f"AES Encrypt Charge: Base {results[0].params['intercept'].value:.03}mC plus {results[0].params['slope'].value:.03} C per byte\n" +
             f"AES Decrypt Charge: Base {results[1].params['intercept'].value:.03} C plus {results[1].params['slope'].value:.03} C per byte\n" +
             f"AES Encrypt Time: Base {results[2].params['intercept'].value:.03} s plus {results[2].params['slope'].value:.03} s per byte\n" +
             f"AES Decrypt Time: Base {results[3].params['intercept'].value:.03} s plus {results[3].params['slope'].value:.03} s per byte\n" +
             f"Flash Write Charge: Base {results[4].params['intercept'].value:.03} C plus {results[4].params['slope'].value:.03} C per byte\n" +
             f"Flash Read Charge: Base {results[5].params['intercept'].value:.03} C plus {results[5].params['slope'].value:.03} C per byte\n" +
             f"Flash Write Time: Base {results[6].params['intercept'].value:.03} s plus {results[6].params['slope'].value:.03} s per byte\n" +
             f"Flash Read Time: Base {results[7].params['intercept'].value:.03} s plus {results[7].params['slope'].value:.03} s per byte\n")
fig2.tight_layout()
fig2.show()

In [103]:
# Save Charge amount list into pickle file
import pickle
pickle.dump(aes_encrypt_charge, open("aes_flash_encrypt_charge.p", "wb"))
pickle.dump(aes_decrypt_charge, open("aes_flash_decrypt_charge.p", "wb"))
pickle.dump(aes_flash_write_charge, open("aes_flash_write_charge.p", "wb"))
pickle.dump(aes_flash_read_charge, open("aes_flash_read_charge.p", "wb"))
pickle.dump(aes_encrypt_time, open("aes_flash_encrypt_time.p", "wb"))
pickle.dump(aes_decrypt_time, open("aes_flash_decrypt_time.p", "wb"))
pickle.dump(aes_flash_write_time, open("aes_flash_write_time.p", "wb"))
pickle.dump(aes_flash_read_time, open("aes_flash_read_time.p", "wb"))

In [98]:
aes = [aes_encrypt_charge, aes_decrypt_charge, aes_flash_write_charge, aes_flash_read_charge, aes_encrypt_time, aes_decrypt_time, aes_flash_write_time, aes_flash_read_time]
for i in aes:
    print(len(i), len(i)*16)

375 6000
375 6000
375 6000
375 6000
375 6000
375 6000
375 6000
375 6000


## Write config file

In [23]:
import json

config = {}
config["name"] = "AES Flash"
config["project_paths"] = [project_path]
config["config_dict"] = config_dict
config["config_dict_plot"] = config_dict_plot
config["analysis"] = {"pins":{2: ["AES Encrypt", "AES Decrypt"], 3: ["Flash Write", "Flash Read"]}, 
                      "result_types": ["Charge", "Time"],
                      "section_types": {"init": [], 
                                        "store": ["AES Encrypt", "Flash Write"],
                                        "load": ["Flash Read", "AES Decrypt"],
                                        "exit": []},
                      "labels": {
                          "Charge": {"x":"Data Size", "x_unit": "byte", "y": "Charge", "y_unit": "C"},
                          "Time": {"x":"Data Size", "x_unit": "byte", "y": "Time", "y_unit": "s"},
                      },
                      "x_step": MBEDTLS_AES_BLOCK_SIZE}

with open("looped_experiment.json", 'w') as config_file:  
    json.dump(config, config_file, indent=4)